# EFA25@DNB
## LLM via Schnittstelle ansprechen und rudimentäres RAG-System aufbauen

Die einzige Bibliothek, die benötigt wird, ist `requests`.

In [ ]:
import requests

Auf der Seite [huggingface.co](https://huggingface.co/) muss ein Konto erstellt werden, um einen Access-Token generieren zu lassen. Dieser Token muss als nächstes eingelesen werden. Es ist zu empfehlen, ihn nicht direkt im Code zu hinterlegen, sondern aus einer extern liegenden Datei einzulesen.

In [ ]:
TOKEN = "hf_xxxxxxx" # Hier Token einlesen

Über `API-URL` ist die Schnittstelle angegeben. Über `MODEL` wird das Sprachmodell bestimmt, in diesem Fall Qwen2.5 mit 72 Milliarden Parametern. 



In [ ]:
API_URL = "https://router.huggingface.co/v1/chat/completions"
MODEL = "Qwen/Qwen2.5-72B-Instruct:together"

### Achtung!
*Das Angebot der Sprachmodelle variiert, insbesondere bei denen, die kostenlos verfügbar sind. Sollte das hier verwendete Qwen-Modell in ein paar Monaten nicht mehr angeboten werden, muss bei Huggingface ein anderes Modell gewählt werden. Möglicherweise sind dann Änderungen am Code notwendig.*



Als nächstes wird die Variable `content` befüllt. Hier befinden sich die Daten, auf die das Sprachmodell zugreifen soll. In diesem Fall sind es (fiktive) Öffnungszeiten der Deutschen Nationalbibliothek ([Link zu den echten Öffnungszeiten](https://www.dnb.de/oeffnungszeiten)).

In [ ]:
content = """Antworte auf Deutsch.
             Deutsche Nationalbibliothek
             Adresse:
                 Deutscher Platz 1
                 04103 Leipzig            
             Öffnungszeiten: 
                 Montag bis Freitag 09:00 Uhr bis 22:00 Uhr
                 Samstag 10:00 Uhr bis 18:00 Uhr
                
             Deutsche Nationalbibliothek 
             Adresse:
                 Adickesallee 1
                 60322 Frankfurt am Main            
             Öffnungszeiten: 
                 Montag bis Freitag 09:00 Uhr bis 16:00 Uhr
                 Samstag 09:00 Uhr bis 13:00 Uhr"""

Alternativ kann auch eine Textdatei eingelesen werden. In `datensets.txt` befinden sich die Namen aller Datensets des DNBLabs (Stand: November 2025) mit ihren Beschreibungen.


In [ ]:
with open("datensets.txt", "r") as f:
    content = f.read()

Die Funktion `llm(query)` ist das Herzstück dieses Codes. Hier wird die Verbindung zur Schnittstelle des gewählten Modells hergestellt. 

Wichtig ist `messages`, wo unter `"role": "system"` der Systemprompt zu finden ist, der in unserem Fall aus dem besteht, was oben unter `content` zu finden ist. Unter `"role": "user"` wird der Prompt des Nutzers übergeben. Alle Daten werden bei der Generierung des Outputs durch das Sprachmodell berücksichtigt.

Darunter finden sich die Parameter `temperature`, `top_p` und `top_k`, über die sich der Grad der Kreativität der Antwort regeln lässt.

In [ ]:
def llm(query):
    response = requests.post(API_URL,
                            headers={"Authorization": f"Bearer {TOKEN}"},
                            json={"model": MODEL,
                                 "messages": [{"role": "system", "content": content},
                                              {"role": "user", "content": query}],
                                 "max_tokens": 500, # Legt die Anzahl der anzuzeigenden Token fest.
                                 "temperature": 0.5, # Regelt Kreativität der Antwort. 1 kreativ, 0 konservativ
                                 "top_p": 0.95, # Berücksichtigt alle Token, die zusammengenommen die Wahrscheinlichkeit p haben.
                                 "top_k": 50}) # Berücksichtigt Anzahl k Token.
    
    response_json = response.json()["choices"][0]["message"]["content"]
    
    return response_json

Beim Aufruf der Funktion wird der Prompt als Parameter übergeben. 

In [ ]:
print(llm("Ich bin am Donnerstag halb 10 abends in Leipzig. Kann ich in der DNB arbeiten?"))
#print(llm("Ich arbeite zum Rauchwarenhandel in den 1920er Jahren. Welche Sets könnten für mich interessant sein?"))

Wer nicht allein die `print()`-Statements nutzen möchte, sondern lieber mehrere Fragen hintereinander stellen möchte, kann alternativ mit einer `while`-Schleife arbeiten. Mit `exit`, `quit` oder `q` als Eingabe wird das Programm beendet.

In [ ]:
while True:    
    user_input = input("Frage: ")
    
    if user_input.lower() in ["exit", "quit", "q"]:
        print("Programm beendet.")
        break
    
    llm_output = llm(user_input)
    print("Antwort:", llm_output)

Über die `while`-Schleife kann man zwar nacheinander Fragen stellen, aber nicht klassisch "chatten". Zwei aufeinanderfolgende Fragen werden nicht in Beziehung miteinander gesetzt, da dem Modell der Kontext der bisherigen Konversation unbekannt ist. Um das zu ändern, müsste man den Inhalt von `user_input` sowie von `llm_output` jedes Mal in einer separaten Liste speichern und über die Variable `content` dem Modell mitgeben.